In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import gurobipy

In [2]:
Elec_Demand = pd.read_csv('Electricity_Demand.csv')
Hydro_Demand = pd.read_csv('Hydrogen_Demand.csv')

Elec_Price = pd.read_csv('Electricity_Prices.csv')
Solar_Supply = pd.read_csv('Solar_Forecast.csv')

In [38]:
# display(Elec_Demand)
# display(Hydro_Demand)
# display(Elec_Price)
# display(Solar_Supply)

In [3]:
# Manipulate data to get it into a convienient form
Solar_Supply['Datetime'] = pd.to_datetime(Solar_Supply['Datetime'])
Solar_Supply['Hour'] = Solar_Supply['Datetime'].dt.hour
cols = ['Datetime', 'Hour', '% of Rated Output']
Solar_Supply = Solar_Supply[cols]
Solar_Supply['MWh'] = Solar_Supply['% of Rated Output']*2

Iteration 1: Part A

In [40]:
# Define Parameters
sE = Solar_Supply['MWh'].to_numpy() # Number of MWs of electricity produced at solar farm for hour i
pE = Elec_Price['Price ($/MWh)'].to_numpy() # Electricity price per MWh at hour i
pH = np.array([10 for i in range(pE.size)]) # Price of hydrogen per kg at hour i
dE = Elec_Demand['Demand (MWh)'].to_numpy() # Electricity demand in MWh at hour i
dH = Hydro_Demand['Demand (kg)'].to_numpy() # Hydrogen demand in kg at hour i

# Define Decision Variables
xE = cp.Variable(pE.size) # Number of MWhs of electricity purchased at hour i
xH = cp.Variable(pE.size) # Number of kgs of hydrogen purchased at hour i
nH = cp.Variable(pE.size) # Number of kgs of hydrogen in inventory at end of hour i

constraints = [xE[0] + sE[0] >= dE[0],                      # Electricity Demand Constraint at hour 1
               xH[0] >= dH[0] + nH[0],                      # Hydrogen Demand Constraint at hour 1
               xE[0] >= 0, xH[0] >= 0, nH[0] >=0]           # Sign constraints at hour 1
for i in range(1,pE.size):
    constraints += [xE[i] + sE[i] >= dE[i],                 # Electricity Demand Constraint at hour 2 to i
                    nH[i-1] + xH[i] >= dH[i] + nH[i],       # Hydrogen Demand Constraint at hour 2 to i
                    xE[i] >= 0, xH[i] >= 0, nH[i] >=0]      # Sign constraints at hour 2 to i

objective = cp.Minimize(pE.T@xE + pH.T@xH)                  # Minimize the cost

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)
print("The optimal value is:", problem.value)
print("The first 5 dual values solutions are:")
print("Electricity Demand Constraint:", problem.constraints[0].dual_value) # Dual value for the Electricity Demand Constraint at hour 1
print("Hydrogen Demand Constraint:", problem.constraints[1].dual_value) # Dual value for the Hydrogen Demand Constraint at hour 1
print("xE Sign Constraint:", problem.constraints[2].dual_value) # Dual value for the xE Sign Constraint at hour 1
print("xH Sign Constraint:", problem.constraints[3].dual_value) # Dual value for the xH Sign Constraint at hour 1
print("nH Sign Constraint:", problem.constraints[4].dual_value) # Dual value for the nH Sign Constraint at hour 1

The optimal value is: 17916.012987517464
The first 5 dual values solutions are:
Electricity Demand Constraint: 22.87
Hydrogen Demand Constraint: 10.0
xE Sign Constraint: -0.0
xH Sign Constraint: -0.0
nH Sign Constraint: -0.0


Iteration 2: Part A (Removed storage as a decision variable)

In [4]:
# Define Parameters
sE = Solar_Supply['MWh'].to_numpy() # Number of MWs of electricity produced at solar farm for hour i
pE = Elec_Price['Price ($/MWh)'].to_numpy() # Electricity price per MWh at hour i
pH = np.array([10 for i in range(pE.size)]) # Price of hydrogen per kg at hour i
dE = Elec_Demand['Demand (MWh)'].to_numpy() # Electricity demand in MWh at hour i
dH = Hydro_Demand['Demand (kg)'].to_numpy() # Hydrogen demand in kg at hour i

# Define Decision Variables
xE = cp.Variable(pE.size) # Number of MWhs of electricity purchased at hour i
xH = cp.Variable(pE.size) # Number of kgs of hydrogen purchased at hour i

constraints = [xE[0] + sE[0] >= dE[0],                              # Electricity Demand Constraint at hour 1
               xH[0] >= dH[0],                                      # Hydrogen Demand Constraint at hour 1
               xE[0] >= 0, xH[0] >= 0]                              # Sign constraints at hour 1
for i in range(1,pE.size):
    constraints += [xE[i] + sE[i] >= dE[i],                         # Electricity Demand Constraint at hour 2 to i
                    sum(xH[0:i+1])-sum(dH[0:i]) >= dH[i],           # Hydrogen Demand Constraint at hour 2 to i
                    xE[i] >= 0, xH[i] >= 0]                         # Sign constraints at hour 2 to i

objective = cp.Minimize(pE.T@xE + pH.T@xH)                          # Minimize the cost

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)
print("The optimal value is:", problem.value)
print("The first 5 dual values solutions are:")
print("Electricity Demand Constraint:", problem.constraints[0].dual_value) # Dual value for the Electricity Demand Constraint at hour 1
print("Hydrogen Demand Constraint:", problem.constraints[1].dual_value) # Dual value for the Hydrogen Demand Constraint at hour 1
print("xE Sign Constraint:", problem.constraints[2].dual_value) # Dual value for the xE Sign Constraint at hour 1
print("xH Sign Constraint:", problem.constraints[3].dual_value) # Dual value for the xH Sign Constraint at hour 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-03
The optimal value is: 17916.012987517457
The first 5 dual values solutions are:
Electricity Demand Constraint: 22.87
Hydrogen Demand Constraint: -0.0
xE Sign Constraint: -0.0
xH Sign Constraint: -0.0


In [5]:
# Define Parameters
sE = Solar_Supply['MWh'].to_numpy() # Number of MWs of electricity produced at solar farm for hour i
pE = Elec_Price['Price ($/MWh)'].to_numpy() # Electricity price per MWh at hour i
pH = np.array([10 for i in range(pE.size)]) # Price of hydrogen per kg at hour i
dE = Elec_Demand['Demand (MWh)'].to_numpy() # Electricity demand in MWh at hour i
dH = Hydro_Demand['Demand (kg)'].to_numpy() # Hydrogen demand in kg at hour i

# Define Decision Variables
xE = cp.Variable(pE.size) # Number of MWhs of electricity purchased at hour i
xH = cp.Variable(pE.size) # Number of kgs of hydrogen purchased at hour i

constraints = []
for i in range(0,pE.size):
    constraints += [xE[i] + sE[i] >= dE[i],                         # Electricity Demand Constraint at hour 2 to i
                    sum(xH[0:i+1])-sum(dH[0:i]) >= dH[i],           # Hydrogen Demand Constraint at hour 2 to i
                    xE[i] >= 0, xH[i] >= 0]                         # Sign constraints at hour 2 to i

objective = cp.Minimize(pE.T@xE + pH.T@xH)                          # Minimize the cost

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)
print("The optimal value is:", problem.value)
print("The first 5 dual values solutions are:")
print("Electricity Demand Constraint:", problem.constraints[0].dual_value) # Dual value for the Electricity Demand Constraint at hour 1
print("Hydrogen Demand Constraint:", problem.constraints[1].dual_value) # Dual value for the Hydrogen Demand Constraint at hour 1
print("xE Sign Constraint:", problem.constraints[2].dual_value) # Dual value for the xE Sign Constraint at hour 1
print("xH Sign Constraint:", problem.constraints[3].dual_value) # Dual value for the xH Sign Constraint at hour 1

The optimal value is: 17916.012987517457
The first 5 dual values solutions are:
Electricity Demand Constraint: 22.87
Hydrogen Demand Constraint: -0.0
xE Sign Constraint: -0.0
xH Sign Constraint: -0.0


In [6]:
Solution_DataFrame = pd.DataFrame()
Solution_DataFrame['Energy Needed'] = dE
Solution_DataFrame['Solar Energy Generated'] = sE
Solution_DataFrame['Energy Purchased'] = xE.value
Solution_DataFrame['Energy Cost at Hour'] = pE
Solution_DataFrame['Hydrogen Needed'] = dH
Solution_DataFrame['Hydrogen Purchased'] = xH.value
Solution_DataFrame['Hydrogen Cost at Hour'] = pH
Solution_DataFrame['Total Cost at Hour'] = (Solution_DataFrame['Energy Purchased'].values * Solution_DataFrame['Energy Cost at Hour'].values
                                            + Solution_DataFrame['Hydrogen Purchased'].values * Solution_DataFrame['Hydrogen Cost at Hour'].values)
Solution_DataFrame['Total Cost'] = Solution_DataFrame['Total Cost at Hour'].cumsum(axis=0)
Solution_DataFrame.index.name = 'Hour'
Solution_DataFrame.index = Solution_DataFrame.index+1
Solution_DataFrame.to_csv('standard output.csv')
# display(Solution_DataFrame)

Iteration 3: Part B Solar Panels (b=2)

In [7]:
# Define Parameters
b = 2 # number of solar panels purchased
Solar_Supply['MWh'] = Solar_Supply['% of Rated Output']
sE = Solar_Supply['MWh'].to_numpy() # Number of MWs of electricity produced at solar farm for hour i
pE = Elec_Price['Price ($/MWh)'].to_numpy() # Electricity price per MWh at hour i
pH = np.array([10 for i in range(pE.size)]) # Price of hydrogen per kg at hour i
dE = Elec_Demand['Demand (MWh)'].to_numpy() # Electricity demand in MWh at hour i
dH = Hydro_Demand['Demand (kg)'].to_numpy() # Hydrogen demand in kg at hour i

# Define Decision Variables
xE = cp.Variable(pE.size) # Number of MWhs of electricity purchased at hour i
xH = cp.Variable(pE.size) # Number of kgs of hydrogen purchased at hour i

constraints = []
for i in range(0,pE.size):
    constraints += [xE[i] + (sE[i]*(2+(0.2*b))) >= dE[i],           # Electricity Demand Constraint at hour 2 to i
                    sum(xH[0:i+1])-sum(dH[0:i]) >= dH[i],           # Hydrogen Demand Constraint at hour 2 to i
                    xE[i] >= 0, xH[i] >= 0]                         # Sign constraints at hour 2 to i

objective = cp.Minimize(pE.T@xE + pH.T@xH)                          # Minimize the cost

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)
print("The optimal value is:", problem.value)
print("The first 5 dual values solutions are:")
print("Electricity Demand Constraint:", problem.constraints[0].dual_value) # Dual value for the Electricity Demand Constraint at hour 1
print("Hydrogen Demand Constraint:", problem.constraints[1].dual_value) # Dual value for the Hydrogen Demand Constraint at hour 1
print("xE Sign Constraint:", problem.constraints[2].dual_value) # Dual value for the xE Sign Constraint at hour 1
print("xH Sign Constraint:", problem.constraints[3].dual_value) # Dual value for the xH Sign Constraint at hour 1

The optimal value is: 17906.248659517456
The first 5 dual values solutions are:
Electricity Demand Constraint: 22.87
Hydrogen Demand Constraint: -0.0
xE Sign Constraint: -0.0
xH Sign Constraint: -0.0


2 extra Solar Panels Bought

In [8]:
Solution_DataFrame = pd.DataFrame()
Solution_DataFrame['Energy Needed'] = dE
Solution_DataFrame['Solar Energy Generated'] = sE*(2+(0.2*b))
Solution_DataFrame['Energy Purchased'] = xE.value
Solution_DataFrame['Energy Cost at Hour'] = pE
Solution_DataFrame['Hydrogen Needed'] = dH
Solution_DataFrame['Hydrogen Purchased'] = xH.value
Solution_DataFrame['Hydrogen Cost at Hour'] = pH
Solution_DataFrame['Total Cost at Hour'] = (Solution_DataFrame['Energy Purchased'].values * Solution_DataFrame['Energy Cost at Hour'].values
                                            + Solution_DataFrame['Hydrogen Purchased'].values * Solution_DataFrame['Hydrogen Cost at Hour'].values)
Solution_DataFrame['Total Cost'] = Solution_DataFrame['Total Cost at Hour'].cumsum(axis=0)
Solution_DataFrame.index.name = 'Hour'
Solution_DataFrame.index = Solution_DataFrame.index+1
Solution_DataFrame.to_csv('2SolarPanelsXtra.csv')
# display(Solution_DataFrame)

Iteration 3: Part B Solar Panels (b=x)

In [9]:
# Define Parameters
Solar_Supply['MWh'] = Solar_Supply['% of Rated Output']
sE = Solar_Supply['MWh'].to_numpy() # Number of MWs of electricity produced at solar farm for hour i
pE = Elec_Price['Price ($/MWh)'].to_numpy() # Electricity price per MWh at hour i
pH = np.array([10 for i in range(pE.size)]) # Price of hydrogen per kg at hour i
dE = Elec_Demand['Demand (MWh)'].to_numpy() # Electricity demand in MWh at hour i
dH = Hydro_Demand['Demand (kg)'].to_numpy() # Hydrogen demand in kg at hour i

# Define Decision Variables
xE = cp.Variable(pE.size) # Number of MWhs of electricity purchased at hour i
xH = cp.Variable(pE.size) # Number of kgs of hydrogen purchased at hour i
b = cp.Variable(1, integer=True) # number of solar panels purchased

constraints = [b >= 0]
for i in range(0,pE.size):
    constraints += [xE[i] + (sE[i]*(2+(0.2*b))) >= dE[i],           # Electricity Demand Constraint at hour 2 to i
                    sum(xH[0:i+1])-sum(dH[0:i]) >= dH[i],           # Hydrogen Demand Constraint at hour 2 to i
                    xE[i] >= 0, xH[i] >= 0]                         # Sign constraints at hour 2 to i

objective = cp.Minimize(pE.T@xE + pH.T@xH + + 2113.5*b)             # Minimize the cost

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)
print("The optimal value is:", problem.value)
print("The optimal number of solar panels to purchase:", b.value)

The optimal value is: 17916.012987517457
The optimal number of solar panels to purchase: [-0.]


x extra Solar Panels Bought

In [11]:
Solution_DataFrame = pd.DataFrame()
Solution_DataFrame['Energy Needed'] = dE
Solution_DataFrame['Solar Energy Generated'] = sE*(2+(0.2*b.value))
Solution_DataFrame['Energy Purchased'] = xE.value
Solution_DataFrame['Energy Cost at Hour'] = pE
Solution_DataFrame['Hydrogen Needed'] = dH
Solution_DataFrame['Hydrogen Purchased'] = xH.value
Solution_DataFrame['Hydrogen Cost at Hour'] = pH
Solution_DataFrame['Total Cost at Hour'] = (Solution_DataFrame['Energy Purchased'].values * Solution_DataFrame['Energy Cost at Hour'].values
                                            + Solution_DataFrame['Hydrogen Purchased'].values * Solution_DataFrame['Hydrogen Cost at Hour'].values)
Solution_DataFrame['Total Cost'] = Solution_DataFrame['Total Cost at Hour'].cumsum(axis=0)
Solution_DataFrame.index.name = 'Hour'
Solution_DataFrame.index = Solution_DataFrame.index+1
Solution_DataFrame.to_csv('xSolarPanelsXtra.csv')
# display(Solution_DataFrame)